<a href="https://colab.research.google.com/github/akhi07rx/CoLab-Torrent-Downloader-Pro/blob/main/CoLab_Torrent_Downloader_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install python3-libtorrent


In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent

In [ ]:
import libtorrent as lt
import time
import re
import os
import pickle
from google.colab import drive, files
from tqdm import tqdm

drive.mount('/content/drive')

ses = lt.session()
ses.listen_on(6881, 6891)

save_path = '/content/drive/MyDrive/Torrent Downloads/'

params = {
    'save_path': save_path,
    'storage_mode': lt.storage_mode_t.storage_mode_sparse,
}

magnet_link_pattern = r'^magnet:\?xt=urn:btih:[a-zA-Z0-9]*'

resume_data_file = os.path.join(save_path, 'resume_data.pickle')

if os.path.exists(resume_data_file):
    with open(resume_data_file, 'rb') as f:
        resume_data = pickle.load(f)
        for data in resume_data:
            ses.add_torrent(data)

while True:
    print("\nMenu:")
    print("1. Enter Magnet link: ")
    print("2. Upload Torrent file: ")
    print("3. Exit")

    choice = input("Select an option (1, 2, or 3): ")

    if choice == '1':
        link = input("Enter the magnet link: ")
        if re.match(magnet_link_pattern, link):
            handle = lt.add_magnet_uri(ses, link, params)
        else:
            print("Invalid magnet link format. Please try again.")
            continue
    elif choice == '2':
        uploaded_files = files.upload()
        if len(uploaded_files) == 0:
            print("No files uploaded. Please try again.")
            continue

        torrent_file_path = list(uploaded_files.values())[0]

        try:
            handle = ses.add_torrent({'ti': lt.torrent_info(torrent_file_path), 'save_path': save_path})
        except Exception as e:
            print(f"Error: {e}")
            continue
    elif choice == '3':
        break
    else:
        print("Invalid choice. Please select a valid option.")
        continue

    print("\nDownloading Metadata...")
    while not handle.has_metadata():
        time.sleep(1)
    print("Got Metadata, Starting Torrent Download...")

    torrent_info = handle.get_torrent_info()
    
    for i in range(torrent_info.num_files()):
        handle.file_priority(i, 1)

    file_size = torrent_info.total_size()
    file_size_mb = file_size / (1024 ** 2)
    file_size_gb = file_size / (1024 ** 3)
    file_size_str = f"{file_size_mb:.2f} MB / {file_size_gb:.2f} GB"
    print(f"File size: {file_size_str}")

    progress_bar = tqdm(total=file_size, unit='B', unit_scale=True, leave=True)

    start_time = time.time()

    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        state_str = ['queued', 'checking', 'downloading metadata', 'downloading', 'finished', 'seeding', 'allocating', 'checking fastresume']
        progress_bar.set_description(state_str[min(s.state, len(state_str)-1)])
        progress_bar.update(s.total_done - progress_bar.n)
        time.sleep(1)

    progress_bar.close()

    end_time = time.time()
    total_time_seconds = end_time - start_time
    total_time_minutes = total_time_seconds / 60

    num_files = torrent_info.num_files()

    print(f"\nFile name: {torrent_info.name()}")
    print(f"File size: {file_size_str}")
    print(f"Total Time: {total_time_seconds:.2f} seconds / {total_time_minutes:.2f} minutes")
    print(f"Number of files: {num_files}")
    print(f"Saved location: {save_path}")

# save resume data
resume_data = []
for handle in ses.get_torrents():
    data = handle.save_resume_data()
    if data:
        resume_data.append({'resume_data': data, 'ti': handle.get_torrent_info(), 'save_path': save_path})
with open(resume_data_file, 'wb') as f:
    pickle.dump(resume_data, f)

while True:
  print("\nFile Options:")
  print("1. Show File Details")
  print("2. Exit to Main Menu")

  file_choice = input("Select an option (1 or 2): ")

  if file_choice == '1':
      for i in range(torrent_info.num_files()):
          file_path = os.path.join(save_path, torrent_info.files().file_path(i))
          if os.path.exists(file_path):
              file_size = os.path.getsize(file_path)
              file_size_mb = file_size / (1024 ** 2)
              file_size_gb = file_size / (1024 ** 3)
              file_size_str = f"{file_size_mb:.2f} MB / {file_size_gb:.2f} GB"
              print(f"\nFile name: {torrent_info.files().file_path(i)}")
              print(f"File size: {file_size_str}")
              print(f"File link: {link}")
  elif file_choice == '2':
      break
  else:
      print("Invalid choice. Please select a valid option.")
      continue



#added file menu
#23